# Intro to SQL

In [28]:
PATH = './assets/datasets/'

## Using sqlite connection

In [29]:
import sqlite3
sqlite_db = 'test_db.sqlite'
conn = sqlite3.connect(sqlite_db)
c = conn.cursor()

In [30]:
c.execute('CREATE TABLE houses (field1 INTEGER PRIMARY KEY, sqft INTEGER, bdrms INTEGER, age INTEGER, price INTEGER);')

# Save (commit) the changes
conn.commit()

OperationalError: table houses already exists

In [31]:
last_sale = (None, 4000, 5, 22, 619000)
c.execute('INSERT INTO houses VALUES (?,?,?,?,?)',last_sale)

# Remember to commit the changes
conn.commit()

In [32]:
recent_sales = [
  (None, 2390, 4, 34, 319000),
  (None, 1870, 3, 14, 289000),
  (None, 1505, 3, 90, 269000),
]

c.executemany('INSERT INTO houses VALUES (?, ?, ?, ?, ?)', recent_sales)

conn.commit()

In [33]:
from numpy import genfromtxt

# import into nparray of ints, then convert to list of lists
data = (genfromtxt(PATH + 'housing-data.csv', dtype='i8',
                    delimiter=',', skip_header=1)).tolist()

In [34]:
data[0:3]

[[2104, 3, 70, 399900], [1600, 3, 28, 329900], [2400, 3, 44, 369000]]

In [35]:
# append a None value to beginning of each sub-list
for d in data:
    d.insert(0, None)

In [36]:
data[0:3]

[[None, 2104, 3, 70, 399900],
 [None, 1600, 3, 28, 329900],
 [None, 2400, 3, 44, 369000]]

In [37]:
# loop through data, running an INSERT on each record (i.e. sublist)
for d in data:
    c.execute('INSERT INTO houses VALUES (?, ?, ?, ?, ?)', d)

conn.commit()

In [38]:
# Similar syntax as before
results = c.execute("SELECT * FROM houses WHERE bdrms = 4")

# Here results is a cursor object - use fetchall() to extract a list
results.fetchall()

[(2, 2390, 4, 34, 319000),
 (6, 2390, 4, 34, 319000),
 (13, 3000, 4, 75, 539900),
 (14, 1985, 4, 61, 299900),
 (19, 1940, 4, 7, 239999),
 (24, 2300, 4, 77, 449900),
 (27, 2609, 4, 5, 499998),
 (28, 3031, 4, 21, 599000),
 (32, 1962, 4, 53, 259900),
 (41, 2040, 4, 75, 314900),
 (43, 1811, 4, 24, 285900),
 (46, 2132, 4, 28, 345000),
 (47, 4215, 4, 66, 549000),
 (48, 2162, 4, 43, 287000),
 (51, 2567, 4, 57, 314000),
 (54, 1852, 4, 64, 299900),
 (57, 2390, 4, 34, 319000),
 (64, 3000, 4, 75, 539900),
 (65, 1985, 4, 61, 299900),
 (70, 1940, 4, 7, 239999),
 (75, 2300, 4, 77, 449900),
 (78, 2609, 4, 5, 499998),
 (79, 3031, 4, 21, 599000),
 (83, 1962, 4, 53, 259900),
 (92, 2040, 4, 75, 314900),
 (94, 1811, 4, 24, 285900),
 (97, 2132, 4, 28, 345000),
 (98, 4215, 4, 66, 549000),
 (99, 2162, 4, 43, 287000),
 (102, 2567, 4, 57, 314000),
 (105, 1852, 4, 64, 299900)]

## Using pandas

In [39]:
import pandas as pd
from pandas.io import sql

In [40]:
import pandas as pd

data = pd.read_csv(PATH + 'housing-data.csv', low_memory=False)
data.head()

,sqft,bdrms,age,price
0,2104,3,70,399900
1,1600,3,28,329900
2,2400,3,44,369000
3,1416,2,49,232000
4,3000,4,75,539900


In [41]:
data.to_sql('houses_pandas',
            con=conn,
            if_exists='replace',
            index=False)

In [42]:
sql.read_sql('select * from houses_pandas limit 5', con=conn)

,sqft,bdrms,age,price
0,2104,3,70,399900
1,1600,3,28,329900
2,2400,3,44,369000
3,1416,2,49,232000
4,3000,4,75,539900


## SQL Operators

## SELECT

```SQL
SELECT
<columns>
FROM
<table>
```

In [43]:
sql.read_sql('select * from houses_pandas limit 10', con=conn)

,sqft,bdrms,age,price
0,2104,3,70,399900
1,1600,3,28,329900
2,2400,3,44,369000
3,1416,2,49,232000
4,3000,4,75,539900
5,1985,4,61,299900
6,1534,3,12,314900
7,1427,3,57,198999
8,1380,3,14,212000
9,1494,3,15,242500


```SQL
SELECT *
```
denotes returns all of the columns.

### We can also select individual columns

```SQL
SELECT
<col1>, <col2>, <coln>
FROM
<table>
```

In [50]:
sql.read_sql('select age, price from houses_pandas limit 10', con=conn)

,age,price
0,70,399900
1,28,329900
2,44,369000
3,49,232000
4,75,539900
5,61,299900
6,12,314900
7,57,198999
8,14,212000
9,15,242500


## Check: Write a query that returns only bedrooms and sq. footage, and price

In [51]:
sql.read_sql('select sqft, bdrms, price from houses_pandas limit 10', con=conn)

,sqft,bdrms,price
0,2104,3,399900
1,1600,3,329900
2,2400,3,369000
3,1416,2,232000
4,3000,4,539900
5,1985,4,299900
6,1534,3,314900
7,1427,3,198999
8,1380,3,212000
9,1494,3,242500


## WHERE

### Where is used to filter the data 

```SQL
SELECT
<columns>
FROM
<table>
WHERE
<condition>
```

### Example

```SQL
SELECT
sqft, bdrms, age, price
FROM houses_pandas
WHERE bdrms = 2 and price < 500000;
```

In [45]:
sql.read_sql('select sqft, bdrms, age, price from houses_pandas where bdrms = 2 and price < 500000', con=conn)

,sqft,bdrms,age,price
0,1416,2,49,232000
1,1320,2,62,299900
2,1888,2,79,255000
3,1839,2,40,349900
4,1664,2,40,368500
5,852,2,70,179900


## Check: Write a query that returns the sqft, bdrms, age for houses older than 60 years.

In [52]:
sql.read_sql('select sqft, bdrms, age from houses_pandas where age > 60', con=conn)

,sqft,bdrms,age
0,2104,3,70
1,3000,4,75
2,1985,4,61
3,2300,4,77
4,1320,2,62
5,1236,3,78
6,1888,2,79
7,2040,4,75
8,3137,3,67
9,4215,4,66


## AGGREGATIONS

- Average (i.e., arithmetic mean)
- Count
- Maximum
- Minimum
- Median
- Mode
- Sum

```SQL
SELECT COUNT(price)
FROM houses_pandas;
```

In [53]:
sql.read_sql('SELECT COUNT(price) FROM houses_pandas', con=conn)

,COUNT(price)
0,47


```SQL
SELECT AVG(sqft), MIN(price), MAX(price)
FROM houses_pandas
WHERE bdrms = 2;
```

In [54]:
sql.read_sql('SELECT AVG(sqft), MIN(price), MAX(price) FROM houses_pandas WHERE bdrms = 2', con=conn)

,AVG(sqft),MIN(price),MAX(price)
0,1496.5,179900,368500


## Check: <br><br> Write a query to find the average price per sq ft for one bedroom houses<br> <br> Write another to find the average price per sq ft for those great than 3 bedrooms

In [59]:
sql.read_sql('SELECT AVG(price/sqft) FROM houses_pandas WHERE bdrms = 1', con=conn)

,AVG(price/sqft)
0,169.0


In [60]:
sql.read_sql('SELECT AVG(price/sqft) FROM houses_pandas WHERE bdrms > 3', con=conn)

,AVG(price/sqft)
0,156.066667


## Independent Practice

Practice querying the **SQLite database** we've created in class using any of the methods you've learnt so far:

- console connection
- python sqlite3 package
- pandas
- BD Browser for SQLite

Practice querying the **PostgreSQL database** you can find at:

```
url: dsi.c20gkj5cvu3l.us-east-1.rds.amazonaws.com
port: 5432
database: dsi
user: dsi_student
password: gastudents
```

using:

- console connection
- python sqlalchemy package
- pandas
- Postico

Questions:

- What's the average price per room for 1 bedroom apartments?
- What's the average price per room for 2 bedrooms apartments?
- What's the most frequent apartment size (in terms of bedrooms)?
- How many are there of that apartment kind?
- What fraction of the total number are of that kind?
- How old is the oldest 3 bedrooms apartment?
- How old is the youngest apartment?
- What's the average age for the whole dataset?
- What's the average age for each bedroom size?

Try to answer all these in SQL.

If you finish, try completing the first sections of [SQL zoo](http://www.sqlzoo.net/).